<h1><b>Web Scrapping Review Hotel dengan SerpAPI</b></h1>

---






*   <h4><b>instal Modul Python Google Search Results<b></h4>

In [42]:
pip install google-search-results

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


*    <h4><b>Import Library</b></h4>

In [43]:
from serpapi import GoogleSearch
from urllib.parse import *

*    <h4><b>Scrapping Local Results</b></h4>
mengambil data 10 objek (hotel) yang memiliki ≥ 100 reviews

In [44]:
params = {
  "engine": "google_maps",
  "q": "mall", # objek yang dicari
  "ll": "@-6.2552348,106.8106399,14.84z", #koordinat tempat sekitar
  "type": "search",
  "api_key": "ea7fc95320e6a3cb7cc5de797549d6be65c5a99b0e4beb4986b4283dec57e863"
}

search = GoogleSearch(params)

lIdx = 0 # indeks yang ditampilkan dari data
lSum = 7 # Batas jumlah data yang ditampilkan
local_results = []

while lIdx  <= lSum:
  results = search.get_dict()

  for Result in results["local_results"]:
    lIdx += 1

    if lIdx <= lSum:

      #seleksi data (jangan ambil data jika jumlah review < 200 )
      if Result["reviews"] < 200:
        lIdx += 1
        continue
        
      # Append data sesuai dengan batas jumlah yang sudah ditentukan
      local_results.append({'Nama': Result["title"],
                           'data_id': Result["data_id"],
                           'total_reviews': Result["reviews"]});

  #menghilangkan pagination pada web, sehingga data yang terambil bisa banyak
  if "next" in results.get("serpapi_pagination", {}):
    # Akan mengubah parameter dari 'GoogleSearch()' dengan isi parameter dari halaman selanjutnya
    search.params_dict.update(dict(parse_qsl(urlsplit(results.get("serpapi_pagination").get("next")).query)))

  else:
    break



In [45]:
for LR in local_results:
  print(LR)

{'Nama': 'Blok M Square', 'data_id': '0x2e69f1ebfd6daa4d:0x8f10b22d03d7c9db', 'total_reviews': 39378}
{'Nama': 'Pacific Place Mall', 'data_id': '0x2e69f151104aee2b:0xc4a608b34b5f03cc', 'total_reviews': 47756}
{'Nama': 'Pondok Indah Mall 1', 'data_id': '0x2e69f1a5f4fbee47:0x682837113c4ea854', 'total_reviews': 79356}
{'Nama': 'Senayan City', 'data_id': '0x2e69f1485e476a65:0x300c043104acc004', 'total_reviews': 58095}
{'Nama': 'The Darmawangsa Square', 'data_id': '0x2e69f1716a1264ff:0x8f2d31415c49a9b8', 'total_reviews': 3211}
{'Nama': 'Melawai Plaza', 'data_id': '0x2e69f16efaa6f6db:0x1cd429e472a9168', 'total_reviews': 2223}
{'Nama': 'Mal Ambasador', 'data_id': '0x2e69f3fbe9c12797:0xffae3870f7ae2fd', 'total_reviews': 36157}


*   <h4><b>Scrapping Reviews</b></h4>
Mengambil 100 data dari masing-masing objek (hotel) sesuai dengan jumlah index yang diinginkan

In [46]:
# data = {'Hotel_name':[],
#         'name':[],
#         'rating':[],
#         'review':[]}

# for LR in local_results:
  
#   params = {
#     "engine": "google_maps_reviews",
#     "data_id": "",
#     "api_key": "ea7fc95320e6a3cb7cc5de797549d6be65c5a99b0e4beb4986b4283dec57e863"
#   }
  
#   params["data_id"] = LR['data_id']

#   search = GoogleSearch(params)

#   lIdx = 0 # Angka indeks dari data
#   lSum = 30 # Batas jumlah data yang ditampilkan

#   while lIdx <= lSum:
#     results = search.get_dict()

#     for Result in results["reviews"]:
#       lIdx += 1

#       if lIdx <= lSum:
#         # Append data sesuai dengan batas jumlah yang sudah ditentukan
#         data['Hotel_name'].append(LR['Nama']);
#         data['name'].append(Result["user"]["name"]);
#         data['rating'].append(Result["rating"]);
#         data['review'].append(Result["snippet"]);

#     #menghilangkan pagination pada web, sehingga data yang terambil bisa banyak
#     if "next" in results.get("serpapi_pagination", {}):
#       # Akan mengubah parameter dari 'GoogleSearch()' dengan isi parameter dari halaman selanjutnya
#       search.params_dict.update(dict(parse_qsl(urlsplit(results.get("serpapi_pagination").get("next")).query)))

#     else:
#       break
    
# review_original = data['review']

data = {'Mall_name': [], 'name': [], 'rating': [], 'review': []}

for LR in local_results:
    params = {
        "engine": "google_maps_reviews",
        "data_id": "",
        "api_key": "ea7fc95320e6a3cb7cc5de797549d6be65c5a99b0e4beb4986b4283dec57e863"
    }

    params["data_id"] = LR['data_id']

    search = GoogleSearch(params)

    lIdx = 0  # Angka indeks dari data
    lSum = 30  # Batas jumlah data yang ditampilkan

    while lIdx <= lSum:
        results = search.get_dict()

        for Result in results.get("reviews", []):
            lIdx += 1

            if lIdx <= lSum:
                # Append data sesuai dengan batas jumlah yang sudah ditentukan
                data['Mall_name'].append(LR['Nama'])
                data['name'].append(Result["user"].get("name", ""))
                data['rating'].append(Result.get("rating", None))
                data['review'].append(Result.get("snippet", ""))

        # menghilangkan pagination pada web, sehingga data yang terambil bisa banyak
        if "next" in results.get("serpapi_pagination", {}):
            # Akan mengubah parameter dari 'GoogleSearch()' dengan isi parameter dari halaman selanjutnya
            search.params_dict.update(dict(parse_qsl(urlsplit(results.get("serpapi_pagination").get("next")).query)))
        else:
            break

review_original = data['review']

*   <h4><b>Export Data Scrapping</b></h4>

In [47]:
import pandas as pd
dfScrap = pd.DataFrame(data)
dfScrap.to_csv('File_CSV/reviewMallJakarta.csv', index = False)

In [48]:
dfScrap

,Mall_name,name,rating,review
0,Blok M Square,Lav,5.0,I spent almost the entire day wandering throug...
1,Blok M Square,Jeremy Mseitif,3.0,Nice Mall with few old shops surrounding it. M...
2,Blok M Square,Agnes Szegedi,5.0,If you want the full Indonesian experience vis...
3,Blok M Square,La Umang,4.0,"The mall is fine, you can find a clothes or ju..."
4,Blok M Square,Dimas Jayasrana,4.0,This is the place if you want to have a sum up...
...,...,...,...,...
205,Mal Ambasador,Wibisana Jusuf,4.0,"Looking for gadgets, cameras, outfits or just ..."
206,Mal Ambasador,Bella Hasibuan,4.0,Old mall but stil comfortable
207,Mal Ambasador,abdrcao,5.0,If you wanna find cool gadgets then tou find t...
208,Mal Ambasador,Asim Gt,4.0,This place is considered one of the oldest mar...


<b>2. Tahap Case Folding</b>


*   <h4><b>Import Library</b></h4>

In [49]:
import re
import string

*   <h4><b>Proses Listing</b></h4>

In [50]:
review_clean = []

for review in review_original:
    # Remove Unicode
    review_test = re.sub(r'[^\x00-\x7F]+', ' ', review)
    # Remove Mentions
    review_test = re.sub(r'@\w+', '', review_test)
    # Lowercase the document
    review_test = review_test.lower()
    # Remove punctuations
    review_test = re.sub(r'[%s]' % re.escape(string.punctuation), ' ', review_test)
    # Remove the numbers
    review_test = re.sub(r'[0-9]', '', review_test)
    # Remove the doubled space
    review_test = re.sub(r'\s{2,}', ' ', review_test)
    
    review_clean.append(review_test)

<b>3. Tahap Tokenizing</b>


*   <h4><b>Import Library</b></h4>

In [51]:
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize, word_tokenize 

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\LENOVO\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


*   <h4><b>Proses Listing</b></h4>

In [52]:
review_tokenized = []

for review in review_clean:
  review_tokenized.append(word_tokenize(review))

<b>4. Tahap Filtering (Stopword Removal)</b>


*   <h4><b>Import Library</b></h4>

In [53]:
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\LENOVO\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

*   <h4><b>Proses Listing</b></h4>

In [54]:
review_filtered = []
listStopword =  set(stopwords.words('english'))

for review in review_tokenized:
  
  review_each_filtered = []
  
  for r in review:
    if r not in listStopword:
        review_each_filtered.append(r)
        
  review_filtered.append(review_each_filtered)

<b>5. Tahap Stemming</b>

menggunakan algoritma <b>SnowballStemmer</b>


In [55]:
from nltk.stem import SnowballStemmer
stemmer = SnowballStemmer("english")

review_stemmed = []
    
for review in review_filtered:
  
  review_each_stemmed = []
  
  for r in review:
    review_each_stemmed.append(stemmer.stem(r))
        
  review_stemmed.append(review_each_stemmed)

<b>6. Export Hasil ke CSV</b>

*   <h4><b>Import Library</b></h4>

In [56]:
import pandas as pd

*   <h4><b>Proses Listing</b></h4>

In [57]:
data_preprocessed = data
data_preprocessed['review'] = review_stemmed

In [58]:
df = pd.DataFrame(data_preprocessed)

In [59]:
df

,Mall_name,name,rating,review
0,Blok M Square,Lav,5.0,"[spent, almost, entir, day, wander, differ, le..."
1,Blok M Square,Jeremy Mseitif,3.0,"[nice, mall, old, shop, surround, import, top,..."
2,Blok M Square,Agnes Szegedi,5.0,"[want, full, indonesian, experi, visit, friday..."
3,Blok M Square,La Umang,4.0,"[mall, fine, find, cloth, food, food, court, b..."
4,Blok M Square,Dimas Jayasrana,4.0,"[place, want, sum, jakarta, youngster, hype, s..."
...,...,...,...,...
205,Mal Ambasador,Wibisana Jusuf,4.0,"[look, gadget, camera, outfit, plain, daili, n..."
206,Mal Ambasador,Bella Hasibuan,4.0,"[old, mall, stil, comfort]"
207,Mal Ambasador,abdrcao,5.0,"[wan, na, find, cool, gadget, tou, find, right..."
208,Mal Ambasador,Asim Gt,4.0,"[place, consid, one, oldest, market, visit, ar..."


In [60]:
df.to_csv('File_CSV/reviewMallJakarta_setelah_preprocessing.csv', index=False)